In [ ]:
!nvidia-smi

Tue Mar 23 06:53:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install torch
!pip install sentencepiece
!pip install tf_sentencepiece
!pip install transformers
!pip install langdetect
!pip install translate-api

In [ ]:
import tensorflow as tf
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
db1=pd.read_excel("/content/drive/MyDrive/Inter IIT - Bridgei2i/Bridgei2i Zip file/Development Data/dev_data_article.xlsx")
db1=db1.drop_duplicates(subset=['Text'],ignore_index=True)    #Removing duplicates

In [ ]:
article=list(db1['Text'])
headline=list(db1['Headline'])

# Preprocessing

Cleaning

In [ ]:
def remove_html(word):
  soup = BeautifulSoup(word, 'lxml')
  html_free = soup.get_text()
  return html_free

def remove_urls(word):
    url_pattern = re.compile(r'https?:\/\/.*[\r\n]*')
    return url_pattern.sub(r'', word)

ft2=[]
lb2=[]
article_px=[]
for x in range(len(article)):
  temp=remove_html(article[x])
  temp=remove_urls(temp)
  idx=temp.find('\n')
  subw=temp[:idx]
  if(len(subw)<50 or len(subw.split(" "))<7):
    article_px.append(temp[idx:])
  else:
    article_px.append(temp)

article_px=[re.sub('\n+','',t) for t in article_px]

for x in range(len(article_px)):
  if(len(article_px[x])>5):
    ft2.append(article_px[x])
    lb2.append(headline[x])

article_pp=ft2
headline_pp=lb2


In [ ]:
assert len(article_pp)==len(headline_pp)

Translation

In [ ]:
import translators as ts

In [ ]:
article_final=[]
headline_final=[]
count=0;

for a in article_pp:
  if(len(a)>5000):
    a=a[:4999]
  if(a!=''):
    article_final.append(ts.google(str(a))) 
    time.sleep(0.5)
  else:
    article_final.append('')
  print(f"Example number {count}")
  count+=1

count=0

for a in headline_pp:
  if(len(a)>5000):
    a=a[:4999]
  if(a!=''):
    headline_final.append(ts.google(str(a))) 
    time.sleep(0.5)
  else:
    headline_final.append('')
  print(f"Example number {count}")
  count+=1




In [ ]:
dff['ctext']=article_final
dff['text']=headline_final
dff.to_csv("Preprocessed_Dataset_Summarization.csv")

# Evaluation (Random 100 articles)






In [ ]:
!pip install -U sentence-transformers
!pip install -U rouge

     |████████████████████████████████| 81kB 8.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.2-cp37-none-any.whl size=114270 sha256=fc22bb9819dbda31fc658e51a2d6dbf428f77377ceb69f65693cad0d5b7b3179
  Stored in directory: /root/.cache/pip/wheels/c7/45/c2/1e55040d016e678dd9378d96b86b7632a4d820f66c60566c05
Successfully built sentence-transformers


In [ ]:
import scipy
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from rouge import Rouge
from statistics import mean
import nltk
nltk.download('punkt')
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
articles=article_final
headlines=headline_final
import random
temp = list(zip(articles, headlines)) 
random.shuffle(temp) 
articles, headlines = zip(*temp)

## T5 with BART layer

In [ ]:
from transformers import T5Tokenizer,T5ForConditionalGeneration

In [ ]:
model=T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/H2_B2I_14/T5")
tokenizer=T5Tokenizer.from_pretrained('t5-base')

In [ ]:
headlines_gen = []
a= 0
for y in articles[:100]:
  inputs = tokenizer.encode("summarize: " + y, return_tensors="pt")
  outputs=model.generate(inputs,length_penalty=2.0, num_beams=4, early_stopping=True)
  headlines_gen.append(tokenizer.decode(outputs[0],skip_special_tokens=True))
  print(f"Example number {a}")
  a+=1

Token indices sequence length is longer than the specified maximum sequence length for this model (922 > 512). Running this sequence through the model will result in indexing errors


Example number 0
Example number 1
Example number 2
Example number 3
Example number 4
Example number 5
Example number 6
Example number 7
Example number 8
Example number 9
Example number 10
Example number 11
Example number 12
Example number 13
Example number 14
Example number 15
Example number 16
Example number 17
Example number 18
Example number 19
Example number 20
Example number 21
Example number 22
Example number 23
Example number 24
Example number 25
Example number 26
Example number 27
Example number 28
Example number 29
Example number 30
Example number 31
Example number 32
Example number 33
Example number 34
Example number 35
Example number 36
Example number 37
Example number 38
Example number 39
Example number 40
Example number 41
Example number 42
Example number 43
Example number 44
Example number 45
Example number 46
Example number 47
Example number 48
Example number 49
Example number 50
Example number 51
Example number 52
Example number 53
Example number 54
Example number 55
Ex

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

/usr/local/lib/python3.7/dist-packages/transformers/models/bart/configuration_bart.py:178: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions.The config can simply be saved and uploaded again to be fixed.
  f"Please make sure the config includes `forced_bos_token_id={self.bos_token_id}` in future versions."


In [ ]:
a=0
headlines_final = []
for x in headlines_gen:
  headlines_final.append(summarizer(x, max_length=50, min_length=10, do_sample=False)[0]['summary_text'])
  print(f"Example Number {a}")
  a+=1
  

Your max_length is set to 50, but you input_length is only 20. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 50, but you input_length is only 21. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Example Number 0


Your max_length is set to 50, but you input_length is only 16. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Example Number 1


Your max_length is set to 50, but you input_length is only 18. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


...

Example Number 96


Your max_length is set to 50, but you input_length is only 19. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Example Number 97


Your max_length is set to 50, but you input_length is only 19. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Example Number 98
Example Number 99


Evaluation

In [ ]:
# Get average document similarity using BERT
model = SentenceTransformer('bert-base-nli-mean-tokens')
sim=[]
# Get a vector for each headlines
for i,j in zip(headlines[:100],headlines_final):
  actual_headline = i
  predicted_headline = j
  actual_headline_embeddings = model.encode(actual_headline)
  predicted_headline_embeddings = model.encode(predicted_headline)
  distance = scipy.spatial.distance.cdist([actual_headline_embeddings], [predicted_headline_embeddings], "cosine")[0]
  print("Similarity Score: %.4f" % (1-distance))
  sim.append(1-distance)


Similarity Score: 0.5415
Similarity Score: 0.3872
Similarity Score: 0.8082
Similarity Score: 0.4605
Similarity Score: 0.8115
Similarity Score: 0.9265
Similarity Score: 0.6262
Similarity Score: 0.8032
Similarity Score: 0.9336
Similarity Score: 0.9480
Similarity Score: 0.8644
Similarity Score: 0.7877
Similarity Score: 0.6009
Similarity Score: 0.5339
Similarity Score: 0.4486
Similarity Score: 0.6341
Similarity Score: 0.6339
Similarity Score: 0.3592
Similarity Score: 0.8174
Similarity Score: 0.5231
Similarity Score: 0.7512
Similarity Score: 0.7257
Similarity Score: 0.9100
Similarity Score: 0.5640
Similarity Score: 0.8832
Similarity Score: 0.6151
Similarity Score: 0.5958
Similarity Score: 0.3396
Similarity Score: 0.6434
Similarity Score: 0.5938
Similarity Score: 0.9267
Similarity Score: 0.8799
Similarity Score: 0.8335
Similarity Score: 0.5760
Similarity Score: 0.5593
Similarity Score: 0.6958
Similarity Score: 0.4633
Similarity Score: 0.7751
Similarity Score: 0.7278
Similarity Score: 0.9631

In [ ]:
# Rouge Scores
rouge = Rouge()
rog=[]
# Headlines Actual and Predicted
for i,j in zip(headlines[:100],headlines_final):
  actual_headline = i
  predicted_headline = j
  rouge_score = rouge.get_scores(actual_headline, predicted_headline)
  rouge_scores = rouge_score[0]['rouge-l']
  rouge_scores
  rog.append(rouge_scores)

In [ ]:
# BLEU Scores
blu=[]  
for i,j in zip(headlines[:100],headlines_final):
  actual_headline = i
  predicted_headline = j
  hypothesis = predicted_headline.split()
  reference = actual_headline.split() 
  references = [reference] # list of references for 1 sentence.
  list_of_references = [references] # list of references for all sentences in corpus.
  list_of_hypotheses = [hypothesis] # list of hypotheses that corresponds to list of references.
  bleu_score = corpus_bleu(list_of_references, list_of_hypotheses)
  bleu_score
  blu.append(bleu_score)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
s=0
u=0
v=0
for x in rog:
  s+=x['p']
  u+=x['r']
  v+=x['f']

In [ ]:
print(f"Mean Similarity == {np.mean(sim)}")
print(f"Mean BLEU == {np.mean(blu)}")
print(f"Mean ROUGE F Score == {v/len(rog)}")
print(f"Mean ROUGE Precision == {s/len(rog)}")
print(f"Mean ROUGE Recall == {u/len(rog)}")

Mean Similarity == 0.6879748035234576
Mean BLEU == 0.3368757104968214
Mean ROUGE F Score == 0.29773620245154564
Mean ROUGE Precision == 0.3662159254816912
Mean ROUGE Recall == 0.2688592351979173


##Pegasus


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [ ]:
model = PegasusForConditionalGeneration.from_pretrained('/content/drive/MyDrive/H2_B2I_14/Pegasus')
tokenizer = PegasusTokenizer.from_pretrained('/content/drive/MyDrive/H2_B2I_14/Pegasus')

In [ ]:
headlines_gen = []
a= 0
for y in articles[:100]:
  temp = tokenizer(y, return_tensors = 'pt',padding=True,truncation=True)
  summ = model.generate(input_ids=temp['input_ids'], 
                        attention_mask=temp['attention_mask'],
                        early_stopping=True)
  pred = tokenizer.decode(summ[0], skip_special_tokens=True)
  headlines_gen.append(pred)
  a+=1
  print(f"Example {a}")

Evaluation

In [ ]:
# Get average document similarity using BERT
model = SentenceTransformer('bert-base-nli-mean-tokens')
sim=[]
# Get a vector for each headlines
for i,j in zip(headlines[:100],headlines_gen):
  if(j==headlines_gen[27]):
    continue
  actual_headline = i
  predicted_headline = j
  actual_headline_embeddings = model.encode(actual_headline)
  predicted_headline_embeddings = model.encode(predicted_headline)
  distance = scipy.spatial.distance.cdist([actual_headline_embeddings], [predicted_headline_embeddings], "cosine")[0]
  print("Similarity Score: %.4f" % (1-distance))
  sim.append(1-distance)

Similarity Score: 0.6546
Similarity Score: 0.3419
Similarity Score: 0.7738
Similarity Score: 0.6645
Similarity Score: 0.8005
Similarity Score: 0.7755
Similarity Score: 0.4647
Similarity Score: 0.9342
Similarity Score: 0.8514
Similarity Score: 0.9330
Similarity Score: 0.9610
Similarity Score: 0.8320
Similarity Score: 0.8432
Similarity Score: 0.5399
Similarity Score: 0.4132
Similarity Score: 0.6750
Similarity Score: 0.5393
Similarity Score: 0.5197
Similarity Score: 0.5322
Similarity Score: 0.6310
Similarity Score: 0.7401
Similarity Score: 0.9142
Similarity Score: 0.7210
Similarity Score: 0.4982
Similarity Score: 0.7289
Similarity Score: 0.8092
Similarity Score: 0.6317
Similarity Score: 0.7327
Similarity Score: 0.8417
Similarity Score: 0.9563
Similarity Score: 0.8930
Similarity Score: 0.6238
Similarity Score: 0.8245
Similarity Score: 0.5429
Similarity Score: 0.7999
Similarity Score: 0.5161
Similarity Score: 0.9657
Similarity Score: 0.9535
Similarity Score: 0.6112
Similarity Score: 0.8943


In [ ]:
import numpy as np

In [ ]:
# Rouge Scores
rouge = Rouge()
rog=[]
# Headlines Actual and Predicted
for i,j in zip(headlines[:100],headlines_gen):
  if(j==headlines_gen[27]):
    continue
  actual_headline = i
  predicted_headline = j
  rouge_score = rouge.get_scores(actual_headline, predicted_headline)
  rouge_scores = rouge_score[0]['rouge-l']
  rouge_scores
  rog.append(rouge_scores)

In [ ]:
# BLEU Scores
blu=[]  
for i,j in zip(headlines[:100],headlines_gen):
  if(j==headlines_gen[27]):
    continue
  actual_headline = i
  predicted_headline = j
  hypothesis = predicted_headline.split()
  reference = actual_headline.split() 
  references = [reference] # list of references for 1 sentence.
  list_of_references = [references] # list of references for all sentences in corpus.
  list_of_hypotheses = [hypothesis] # list of hypotheses that corresponds to list of references.
  bleu_score = corpus_bleu(list_of_references, list_of_hypotheses)
  bleu_score
  blu.append(bleu_score)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
s=0
u=0
v=0
for x in rog:
  s+=x['p']
  u+=x['r']
  v+=x['f']

In [ ]:
headlines_gen

In [ ]:
print(f"Mean Similarity == {np.mean(sim)}")
print(f"Mean BLEU == {np.mean(blu)}")
print(f"Mean ROUGE F Score == {v/len(rog)}")
print(f"Mean ROUGE Precision == {s/len(rog)}")
print(f"Mean ROUGE Recall == {u/len(rog)}")

Mean Similarity == 0.7253841656670601
Mean BLEU == 0.3353949739551357
Mean ROUGE F Score == 0.33432094135389123
Mean ROUGE Precision == 0.516856264008956
Mean ROUGE Recall == 0.2587602374777877


## T5

In [ ]:
from transformers import T5Tokenizer,T5ForConditionalGeneration

In [ ]:
model=T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/H2_B2I_14/T5")
tokenizer=T5Tokenizer.from_pretrained('t5-base')

In [ ]:
headlines_gen = []
a= 0
for y in articles[:100]:
  inputs = tokenizer.encode("summarize: " + y, return_tensors="pt")
  outputs=model.generate(inputs,length_penalty=2.0, num_beams=4, early_stopping=True)
  headlines_gen.append(tokenizer.decode(outputs[0],skip_special_tokens=True))
  print(f"Example number {a}")
  a+=1

Evaluation

In [ ]:
# Get average document similarity using BERT
model = SentenceTransformer('bert-base-nli-mean-tokens')
sim=[]
# Get a vector for each headlines
for i,j in zip(headlines[:100],headlines_gen):
  actual_headline = i
  predicted_headline = j
  actual_headline_embeddings = model.encode(actual_headline)
  predicted_headline_embeddings = model.encode(predicted_headline)
  distance = scipy.spatial.distance.cdist([actual_headline_embeddings], [predicted_headline_embeddings], "cosine")[0]
  print("Similarity Score: %.4f" % (1-distance))
  sim.append(1-distance)

Similarity Score: 0.5350
Similarity Score: 0.3810
Similarity Score: 0.9932
Similarity Score: 0.5114
Similarity Score: 0.8130
Similarity Score: 0.9959
Similarity Score: 0.6097
Similarity Score: 0.7791
Similarity Score: 0.9454
Similarity Score: 0.9877
Similarity Score: 1.0000
Similarity Score: 0.7645
Similarity Score: 0.8441
Similarity Score: 0.5339
Similarity Score: 0.4725
Similarity Score: 0.6602
Similarity Score: 0.6403
Similarity Score: 0.4910
Similarity Score: 0.8232
Similarity Score: 0.5231
Similarity Score: 0.7695
Similarity Score: 0.7292
Similarity Score: 0.9066
Similarity Score: 0.7613
Similarity Score: 0.8822
Similarity Score: 0.6173
Similarity Score: 0.4779
Similarity Score: 0.3653
Similarity Score: 0.6353
Similarity Score: 0.6688
Similarity Score: 0.9444
Similarity Score: 0.8601
Similarity Score: 0.8689
Similarity Score: 0.5760
Similarity Score: 0.5784
Similarity Score: 0.6803
Similarity Score: 0.4763
Similarity Score: 0.8098
Similarity Score: 0.7278
Similarity Score: 0.9638


In [ ]:
# Rouge Scores
rouge = Rouge()
rog=[]
# Headlines Actual and Predicted
for i,j in zip(headlines[:100],headlines_gen):
  actual_headline = i
  predicted_headline = j
  rouge_score = rouge.get_scores(actual_headline, predicted_headline)
  rouge_scores = rouge_score[0]['rouge-l']
  rouge_scores
  rog.append(rouge_scores)

In [ ]:
# BLEU Scores
blu=[]  
for i,j in zip(headlines[:100],headlines_gen):
  actual_headline = i
  predicted_headline = j
  hypothesis = predicted_headline.split()
  reference = actual_headline.split() 
  references = [reference] # list of references for 1 sentence.
  list_of_references = [references] # list of references for all sentences in corpus.
  list_of_hypotheses = [hypothesis] # list of hypotheses that corresponds to list of references.
  bleu_score = corpus_bleu(list_of_references, list_of_hypotheses)
  bleu_score
  blu.append(bleu_score)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
s=0
u=0
v=0
for x in rog:
  s+=x['p']
  u+=x['r']
  v+=x['f']

In [ ]:
print(f"Mean Similarity == {np.mean(sim)}")
print(f"Mean BLEU == {np.mean(blu)}")
print(f"Mean ROUGE F Score == {v/len(rog)}")
print(f"Mean ROUGE Precision == {s/len(rog)}")
print(f"Mean ROUGE Recall == {u/len(rog)}")

Mean Similarity == 0.7013707657051697
Mean BLEU == 0.396197319945035
Mean ROUGE F Score == 0.3173455440182698
Mean ROUGE Precision == 0.34140954388374956
Mean ROUGE Recall == 0.31544321098578054


In [ ]:
headlines_gen

['Digitisation is one of the key buzzwords in the post-Covid world',
 'increase in tolerance limit up to 120 per cent for real estate developers selling residential units meeting certain conditions',
 'Jonas Lossl leaves Everton to rejoin first club Midtjyll',
 'cotton finished moderately higher as corn exports were reputed to be the highest on record',
 'UNICEF enters long-term supply agreement for AstraZeneca/O',
 'the causal effects of long-term PM2.5 exposure on COVID-19 in India:',
 'PRIORIZE Initiative aligns social protection with climate adaptation in Mozambique.',
 'this paper aims to provide companies and financial organizations with a common understanding of climate-related',
 'diabetes during pregnancy linked to heart disease risk: study suggests diabetes during pregnancy linked to heart disease risk',
 'banks closed, ATMs shut down in Myanmar following military coup: reports. ATMs shut',
 'not risking it: Children and youth in climate adaptation and disaster risk reduction

# Pegasus with BART Layer

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [ ]:
model = PegasusForConditionalGeneration.from_pretrained('/content/drive/MyDrive/H2_B2I_14/Pegasus')
tokenizer = PegasusTokenizer.from_pretrained('/content/drive/MyDrive/H2_B2I_14/Pegasus')

In [ ]:
headlines_gen = []
a= 0
for y in articles[:100]:
  temp = tokenizer(y, return_tensors = 'pt',padding=True,truncation=True)
  summ = model.generate(input_ids=temp['input_ids'], 
                        attention_mask=temp['attention_mask'],
                        early_stopping=True)
  pred = tokenizer.decode(summ[0], skip_special_tokens=True)
  headlines_gen.append(pred)
  a+=1
  print(f"Example {a}")

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")

/usr/local/lib/python3.7/dist-packages/transformers/models/bart/configuration_bart.py:178: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions.The config can simply be saved and uploaded again to be fixed.
  f"Please make sure the config includes `forced_bos_token_id={self.bos_token_id}` in future versions."


In [ ]:
a=0
headlines_final = []
for x in headlines_gen:
  headlines_final.append(summarizer(x, max_length=50, min_length=10, do_sample=False)[0]['summary_text'])
  print(f"Example Number {a}")
  a+=1

Your max_length is set to 50, but you input_length is only 38. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 50, but you input_length is only 33. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Example Number 0


Your max_length is set to 50, but you input_length is only 34. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Example Number 1


Your max_length is set to 50, but you input_length is only 39. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


...

Example Number 96


Your max_length is set to 50, but you input_length is only 37. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Example Number 97


Your max_length is set to 50, but you input_length is only 41. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Example Number 98
Example Number 99


Evaluation

In [ ]:
# Get average document similarity using BERT
model = SentenceTransformer('bert-base-nli-mean-tokens')
sim=[]
# Get a vector for each headlines
for i,j in zip(headlines[:100],headlines_final):
  actual_headline = i
  predicted_headline = j
  actual_headline_embeddings = model.encode(actual_headline)
  predicted_headline_embeddings = model.encode(predicted_headline)
  distance = scipy.spatial.distance.cdist([actual_headline_embeddings], [predicted_headline_embeddings], "cosine")[0]
  print("Similarity Score: %.4f" % (1-distance))
  sim.append(1-distance)

Similarity Score: 0.5960
Similarity Score: 0.3427
Similarity Score: 0.7849
Similarity Score: 0.6974
Similarity Score: 0.9981
Similarity Score: 0.7867
Similarity Score: 0.4814
Similarity Score: 0.8637
Similarity Score: 0.8404
Similarity Score: 0.9331
Similarity Score: 0.9710
Similarity Score: 0.8174
Similarity Score: 0.8269
Similarity Score: 0.5375
Similarity Score: 0.4119
Similarity Score: 0.6337
Similarity Score: 0.5450
Similarity Score: 0.4678
Similarity Score: 0.5193
Similarity Score: 0.5553
Similarity Score: 0.7113
Similarity Score: 0.8891
Similarity Score: 0.7265
Similarity Score: 0.4518
Similarity Score: 0.7597
Similarity Score: 0.8092
Similarity Score: 0.6268
Similarity Score: 0.5622
Similarity Score: 0.7601
Similarity Score: 0.8256
Similarity Score: 0.9613
Similarity Score: 0.8884
Similarity Score: 0.6162
Similarity Score: 0.8157
Similarity Score: 0.4736
Similarity Score: 0.8085
Similarity Score: 0.5017
Similarity Score: 0.9426
Similarity Score: 0.9020
Similarity Score: 0.5927


In [ ]:
# Rouge Scores
rouge = Rouge()
rog=[]
# Headlines Actual and Predicted
for i,j in zip(headlines[:100],headlines_final):
  actual_headline = i
  predicted_headline = j
  rouge_score = rouge.get_scores(actual_headline, predicted_headline)
  rouge_scores = rouge_score[0]['rouge-l']
  rouge_scores
  rog.append(rouge_scores)

In [ ]:
# BLEU Scores
blu=[]  
for i,j in zip(headlines[:100],headlines_final):
  actual_headline = i
  predicted_headline = j
  hypothesis = predicted_headline.split()
  reference = actual_headline.split() 
  references = [reference] # list of references for 1 sentence.
  list_of_references = [references] # list of references for all sentences in corpus.
  list_of_hypotheses = [hypothesis] # list of hypotheses that corresponds to list of references.
  bleu_score = corpus_bleu(list_of_references, list_of_hypotheses)
  bleu_score
  blu.append(bleu_score)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
s=0
u=0
v=0
for x in rog:
  s+=x['p']
  u+=x['r']
  v+=x['f']

In [ ]:
print(f"Mean Similarity == {np.mean(sim)}")
print(f"Mean BLEU == {np.mean(blu)}")
print(f"Mean ROUGE F Score == {v/len(rog)}")
print(f"Mean ROUGE Precision == {s/len(rog)}")
print(f"Mean ROUGE Recall == {u/len(rog)}")

Mean Similarity == 0.7035332402018262
Mean BLEU == 0.33411966674241456
Mean ROUGE F Score == 0.31878765232843814
Mean ROUGE Precision == 0.4761153991011309
Mean ROUGE Recall == 0.25136863820957966
